In [1]:
import nltk

In [2]:
def gender_features(word):
    return {'last_letter': word[-1]}
gender_features('Shrek')

{'last_letter': 'k'}

In [3]:
nltk.download('names')

[nltk_data] Downloading package names to /home/yasmin/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [4]:
from nltk.corpus import names
labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
                 [(name, 'female') for name in names.words('female.txt')])
import random
random.shuffle(labeled_names)

In [5]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [6]:
classifier.classify(gender_features('Neo'))

'male'

In [7]:
classifier.classify(gender_features('Trinity'))

'female'

In [8]:
print (nltk.classify.accuracy (classifier, test_set))

0.754


Esta lista mostra que os nomes no conjunto de treinamento que terminam em "a" são 33 vezes mais femininos do que masculinos, mas os nomes que terminam em "k" são 32 vezes mais masculinos do que femininos. Essas taxas são conhecidas como taxas de verossimilhança

In [9]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'k'              male : female =     46.4 : 1.0
             last_letter = 'a'            female : male   =     35.7 : 1.0
             last_letter = 'f'              male : female =     15.9 : 1.0
             last_letter = 'p'              male : female =     11.8 : 1.0
             last_letter = 'v'              male : female =     11.2 : 1.0


In [10]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

In [11]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [12]:
gender_features2('John') 

{'first_letter': 'j',
 'last_letter': 'n',
 'count(a)': 0,
 'has(a)': False,
 'count(b)': 0,
 'has(b)': False,
 'count(c)': 0,
 'has(c)': False,
 'count(d)': 0,
 'has(d)': False,
 'count(e)': 0,
 'has(e)': False,
 'count(f)': 0,
 'has(f)': False,
 'count(g)': 0,
 'has(g)': False,
 'count(h)': 1,
 'has(h)': True,
 'count(i)': 0,
 'has(i)': False,
 'count(j)': 1,
 'has(j)': True,
 'count(k)': 0,
 'has(k)': False,
 'count(l)': 0,
 'has(l)': False,
 'count(m)': 0,
 'has(m)': False,
 'count(n)': 1,
 'has(n)': True,
 'count(o)': 1,
 'has(o)': True,
 'count(p)': 0,
 'has(p)': False,
 'count(q)': 0,
 'has(q)': False,
 'count(r)': 0,
 'has(r)': False,
 'count(s)': 0,
 'has(s)': False,
 'count(t)': 0,
 'has(t)': False,
 'count(u)': 0,
 'has(u)': False,
 'count(v)': 0,
 'has(v)': False,
 'count(w)': 0,
 'has(w)': False,
 'count(x)': 0,
 'has(x)': False,
 'count(y)': 0,
 'has(y)': False,
 'count(z)': 0,
 'has(z)': False}

In [13]:
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.744


O conjunto de treinamento é usado para treinar o modelo e o conjunto dev-test é usado para realizar a análise de erros. O conjunto de teste serve em nossa avaliação final do sistema. Por razões discutidas abaixo, é importante que empreguemos um conjunto de teste de desenvolvimento separado para análise de erros, em vez de apenas usar o conjunto de teste. 

In [14]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

In [15]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.776


In [16]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

Podemos, então, examinar casos de erro individuais em que o modelo previu o rótulo errado e tentar determinar quais informações adicionais permitiriam que ele tomasse a decisão certa (ou quais informações existentes o estão induzindo a tomar a decisão errada). O conjunto de recursos pode então ser ajustado de acordo. O classificador de nomes que construímos gera cerca de 100 erros no corpus dev-test

In [17]:
for (tag, guess, name) in sorted(errors):
     print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

correct=female   guess=male     name=Aeriell                       
correct=female   guess=male     name=Alix                          
correct=female   guess=male     name=Allison                       
correct=female   guess=male     name=Arabel                        
correct=female   guess=male     name=Ardeen                        
correct=female   guess=male     name=Ardis                         
correct=female   guess=male     name=Arlen                         
correct=female   guess=male     name=Astrix                        
correct=female   guess=male     name=Aurel                         
correct=female   guess=male     name=Averil                        
correct=female   guess=male     name=Ayn                           
correct=female   guess=male     name=Beau                          
correct=female   guess=male     name=Beilul                        
correct=female   guess=male     name=Berget                        
correct=female   guess=male     name=Brier      

In [18]:
def gender_features(word):
    return {'suffix1': word[-1:],
           'suffix2': word[-2:]}

In [19]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.785


In [20]:
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/yasmin/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [21]:

from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

In [22]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document): 
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [23]:
print(document_features(movie_reviews.words('pos/cv957_8737.txt')))

{'contains(,)': True, 'contains(the)': True, 'contains(.)': True, 'contains(a)': True, 'contains(and)': True, 'contains(of)': True, 'contains(to)': True, "contains(')": True, 'contains(is)': True, 'contains(in)': True, 'contains(s)': True, 'contains(")': True, 'contains(it)': True, 'contains(that)': True, 'contains(-)': True, 'contains())': True, 'contains(()': True, 'contains(as)': True, 'contains(with)': True, 'contains(for)': True, 'contains(his)': True, 'contains(this)': True, 'contains(film)': False, 'contains(i)': False, 'contains(he)': True, 'contains(but)': True, 'contains(on)': True, 'contains(are)': True, 'contains(t)': False, 'contains(by)': True, 'contains(be)': True, 'contains(one)': True, 'contains(movie)': True, 'contains(an)': True, 'contains(who)': True, 'contains(not)': True, 'contains(you)': True, 'contains(from)': True, 'contains(at)': False, 'contains(was)': False, 'contains(have)': True, 'contains(they)': True, 'contains(has)': True, 'contains(her)': False, 'conta

In [24]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set)) 
classifier.show_most_informative_features(5)

0.75
Most Informative Features
   contains(outstanding) = True              pos : neg    =     11.3 : 1.0
         contains(mulan) = True              pos : neg    =      8.9 : 1.0
   contains(wonderfully) = True              pos : neg    =      8.5 : 1.0
         contains(damon) = True              pos : neg    =      7.8 : 1.0
        contains(seagal) = True              neg : pos    =      7.4 : 1.0


In [25]:
nltk.download('brown')

[nltk_data] Downloading package brown to /home/yasmin/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [26]:
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

In [27]:
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
print(common_suffixes)

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


In [28]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
         features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

In [29]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.6270512182993535

In [30]:
classifier.classify(pos_features('cats'))

'NNS'

Aqui, podemos ver que o classificador começa verificando se uma palavra termina com uma vírgula - em caso afirmativo, ele receberá a tag especial "," . Em seguida, o classificador verifica se a palavra termina em "o" , caso em que é quase certo um determinante. Este "sufixo" é usado no início da árvore de decisão porque a palavra "o" é muito comum. Continuando, o classificador verifica se a palavra termina em "s". Em caso afirmativo, é mais provável que receba a marca verbal VBZ (a menos que seja a palavra "é", que tem uma marca especial BEZ ), e se não, então é mais provável que seja um substantivo (a menos que seja a marca de pontuação ".") . O argumento apenas exibe a parte superior da árvore de decisão.



In [31]:
print (classifier.pseudocode (depth = 4))

if endswith(the) == False: 
  if endswith(,) == False: 
    if endswith(s) == False: 
      if endswith(.) == False: return '.'
      if endswith(.) == True: return '.'
    if endswith(s) == True: 
      if endswith(is) == False: return 'PP$'
      if endswith(is) == True: return 'BEZ'
  if endswith(,) == True: return ','
if endswith(the) == True: return 'AT'



In [33]:
def pos_features(sentence, i):
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features

In [34]:
pos_features(brown.sents()[0], 8)

{'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion', 'prev-word': 'an'}

In [35]:
brown.sents()[0]

['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of',
 "Atlanta's",
 'recent',
 'primary',
 'election',
 'produced',
 '``',
 'no',
 'evidence',
 "''",
 'that',
 'any',
 'irregularities',
 'took',
 'place',
 '.']

Um classificador de classe gramatical cujo detector de características examina o contexto no qual uma palavra aparece para determinar qual etiqueta de classe gramatical deve ser atribuída. Em particular, a identidade da palavra anterior é incluída como um recurso.

In [36]:
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append( (pos_features(untagged_sent, i), tag))
        
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

nltk.classify.accuracy(classifier, test_set)

0.7891596220785678

Uma estratégia de classificação de sequência, conhecida como classificação consecutiva ou classificação de sequência gananciosa , é encontrar o rótulo de classe mais provável para a primeira entrada e, em seguida, usar essa resposta para ajudar a encontrar o melhor rótulo para a próxima entrada. O processo pode então ser repetido até que todas as entradas tenham sido rotuladas.

In [38]:
 def pos_features(sentence, i, history):
    features = {"suffix(1)": sentence[i][-1:],
                 "suffix(2)": sentence[i][-2:],
                 "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
        features["prev-tag"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
        features["prev-tag"] = history[i-1]
    return features

class ConsecutivePosTagger(nltk.TaggerI): 

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)
print(tagger.evaluate(test_sents))


0.7980528511821975


A segmentação de frases pode ser vista como uma tarefa de classificação para pontuação: sempre que encontramos um símbolo que possivelmente poderia encerrar uma frase, como um ponto final ou um ponto de interrogação, temos que decidir se ele termina a frase anterior.

In [40]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /home/yasmin/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

A primeira etapa é obter alguns dados que já foram segmentados em frases e convertê-los em uma forma adequada para extrair recursos

In [41]:
sents = nltk.corpus.treebank_raw.sents()
tokens = []
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset-1)

Aqui, tokens é uma lista mesclada de tokens de frases individuais e limites é um conjunto que contém os índices de todos os tokens de limite de frase. Em seguida, precisamos especificar os recursos dos dados que serão usados ​​para decidir se a pontuação indica um limite de frase

In [43]:
 def punct_features(tokens, i):
    return {'next-word-capitalized': tokens[i+1][0].isupper(),
             'prev-word': tokens[i-1].lower(),
             'punct': tokens[i],
             'prev-word-is-one-char': len(tokens[i-1]) == 1}

In [44]:
featuresets = [(punct_features(tokens, i), (i in boundaries))
                for i in range(1, len(tokens)-1)
                if tokens[i] in '.?!']
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.936026936026936

In [45]:
featuresets

[({'next-word-capitalized': False,
   'prev-word': 'nov',
   'punct': '.',
   'prev-word-is-one-char': False},
  False),
 ({'next-word-capitalized': True,
   'prev-word': '29',
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': True,
   'prev-word': 'mr',
   'punct': '.',
   'prev-word-is-one-char': False},
  False),
 ({'next-word-capitalized': True,
   'prev-word': 'n',
   'punct': '.',
   'prev-word-is-one-char': True},
  False),
 ({'next-word-capitalized': False,
   'prev-word': 'group',
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': True,
   'prev-word': '.',
   'punct': '.',
   'prev-word-is-one-char': True},
  False),
 ({'next-word-capitalized': False,
   'prev-word': 'conglomerate',
   'punct': '.',
   'prev-word-is-one-char': False},
  True),
 ({'next-word-capitalized': True,
   'prev-word': '.',
   'punct': '.',
   'prev-word-is-one-char': True},
  False),
 ({'next-word-capitalized': True,
   'pr

In [46]:
#Segmentador de frases baseado em classificação
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if word in '.?!' and classifier.classify(punct_features(words, i)) == True:
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        sents.append(words[start:])
    return sents

In [48]:
#Identificando Tipos de Ato de Diálogo
nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts () [: 10000]
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
         features['contains({})'.format(word.lower())] = True
    return features
featuresets = [(dialogue_act_features(post.text), post.get('class'))
                for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

[nltk_data] Downloading package nps_chat to /home/yasmin/nltk_data...
[nltk_data]   Unzipping corpora/nps_chat.zip.


0.667


Extrator de recurso "Recognizing Text Entailment". A classe RTEFeatureExtractor cria um pacote de palavras para o texto e a hipótese após descartar algumas palavras irrelevantes e, em seguida, calcula a sobreposição e a diferença.

Esses recursos indicam que todas as palavras importantes na hipótese estão contidas no texto e, portanto, há alguma evidência para rotular isso como Verdadeiro .

O módulo nltk.classify.rte_classify atinge um pouco mais de 58% de precisão nos dados de teste RTE combinados usando métodos como esses. Embora esse número não seja muito impressionante, ele exige um esforço significativo e mais processamento linguístico para obter resultados muito melhores.

In [50]:
nltk.download('rte')
def rte_features(rtepair):
    extractor = nltk.RTEFeatureExtractor(rtepair)
    features = {}
    features['word_overlap'] = len(extractor.overlap('word'))
    features['word_hyp_extra'] = len(extractor.hyp_extra('word'))
    features['ne_overlap'] = len(extractor.overlap('ne'))
    features['ne_hyp_extra'] = len(extractor.hyp_extra('ne'))
    return features

rtepair = nltk.corpus.rte.pairs(['rte3_dev.xml'])[33]
extractor = nltk.RTEFeatureExtractor(rtepair)
print(extractor.text_words)

print(extractor.hyp_words)

print(extractor.overlap('word'))

print(extractor.overlap('ne'))

print(extractor.hyp_extra('word'))


[nltk_data] Downloading package rte to /home/yasmin/nltk_data...


{'four', 'Soviet', 'Asia', 'Co', 'was', 'fight', 'Shanghai', 'fledgling', 'meeting', 'republics', 'Parviz', 'operation', 'association', 'former', 'Davudi', 'together', 'at', 'central', 'representing', 'Iran', 'Russia', 'binds', 'Organisation', 'SCO', 'China', 'terrorism.', 'that'}
{'SCO.', 'member', 'China'}
set()
{'China'}
{'member'}


[nltk_data]   Unzipping corpora/rte.zip.
